# Qiskit Modules Demo

In [ ]:
!python -m pip install --upgrade pip

In [ ]:
!python -m pip install qiskit qiskit-aer pylatexenc matplotlib sympy graphviz qiskit-ibm-runtime seaborn --upgrade

In [ ]:
# Version checking:
import qiskit, qiskit_aer, pylatexenc, matplotlib, sympy, graphviz, qiskit_ibm_runtime, seaborn
print(qiskit.__version__)
print(qiskit_aer.__version__)
print(pylatexenc.__version__)
print(matplotlib.__version__)
print(sympy.__version__)
print(graphviz.__version__)
print(qiskit_ibm_runtime.__version__)
print(seaborn.__version__)

In [ ]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/' 
# Replace with the path to your Graphviz binary (get from https://graphviz.org/download/)

## 1) Importing Dependencies

In [ ]:
# Core circuit
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit.circuit.library import PauliEvolutionGate

# Quantum info
from qiskit.quantum_info import Statevector, SparsePauliOp, Operator, DensityMatrix

# Converters
from qiskit.converters import circuit_to_dag, dag_to_circuit

# Transpiler + Target
from qiskit.transpiler import generate_preset_pass_manager, Target, InstructionProperties
from qiskit.circuit.library import RXGate, RZGate, CXGate, Measure

# Visualization
from qiskit.visualization import plot_histogram, pass_manager_drawer, plot_state_city, plot_state_hinton, plot_gate_map, plot_state_paulivec, plot_coupling_map, plot_error_map, plot_circuit_layout

# Qiskit IBM Runtime
from qiskit_ibm_runtime import EstimatorV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeWashingtonV2

# Qiskit-Aer
from qiskit_aer import AerSimulator

# Primitives
from qiskit.primitives import StatevectorSampler, StatevectorEstimator

import numpy as np

## 2) Build a Parameterized Physical Model (Simulating 4-Qubit Heisenberg Hamiltonian Evolution) 

The Heisenberg Hamiltonian describes how quantum spins interact with one another in a many-body system. It is a fundamental model in condensed matter physics and can be used to simulate magnetic materials, quantum phases of matter, and certain chemical systems.

In simple terms: Any physical quantum system starts off in a specific initial state/configuration (spin up/down is $|0\rangle$/$|1\rangle$). Then the Hamiltonian (H) describes how the system evolves over time (t) to the final state/configuration of the system.

We will simulate a simple XX + ZZ interaction on 4 qubits.

The Hamiltonian matrix for this would be:
$H = \sum_{i=0}^{2} \left( X_i X_{i+1} + Z_i Z_{i+1} \right)$

(Which describes how the system evolves over time, assuming adjacent qubits interact by bit/phase flips alone)

Note for later: After simulating the evolution of the system, we can obtain the average energy level of the quantum system by finding the expectation value of the Hamiltonian (for the state at time t).

In [ ]:
n = 4
t = Parameter("t")

# Build Hamiltonian using SparsePauliOp (memory efficient)
paulis = []
for i in range(n - 1):
    xx = ["I"] * n
    zz = ["I"] * n
    xx[i] = "X"
    xx[i+1] = "X"
    zz[i] = "Z"
    zz[i+1] = "Z"
    
    paulis.append(("".join(xx), 1.0))
    paulis.append(("".join(zz), 1.0))

H = SparsePauliOp.from_list(paulis)

H

So We have successfully created the Hamiltonian $H = XXII + ZZII + IXXI + IZZI + IIXX + IIZZ$

## 3) Building the Quantum Circuit Corresponding to this Hamiltonian

We now create a quantum circuit which implements this Hamiltonian Evolution.

In [ ]:
evo_gate = PauliEvolutionGate(H, time=t) # For now, we leave t (time over which the system evoloves) as a parameter 

qc = QuantumCircuit(n)
qc.h(0)
qc.cx(0,1)
qc.append(evo_gate, range(n))
qc.measure_all()

qc.draw("mpl")

## 4) Assigning Time Parameter and Simulating the Circuit (using State vectors)

We have a specific circuit, but to actually run the simulation (execute the circuit), we need to assign a value to the time parameter (t) 

In [ ]:
qc_assigned = qc.assign_parameters({t: 0.7}) # Assigning t = 0.7 before executing the circuit

qc_no_meas = qc_assigned.remove_final_measurements(inplace=False)
qc_no_meas.draw("mpl")

In [ ]:
# Obtaining the statevector of the circuit
sv = Statevector.from_instruction(qc_no_meas)
sv

In [ ]:
sv.draw('latex') # Using quantum_info methods to see the symbolic expression

* So far: We have simulated the evolution of the quantum system's state over time t.
* Now: We want to estimate the average energy of the quantum system after evolving for time t (What the theoretical value *should* be).
This is so that later, when we actually execute the circuit on some hardware backend, we can verify if our answers are correct.

In [ ]:
estimator = StatevectorEstimator() # Used to estimate the Hamiltonian (energy level) using the Statevector (without actually running the circuit)

job = estimator.run([(qc_no_meas, H)])
result = job.result()

result[0].data.evs

So we see at `t=0.7`, the average energy of the quantum system is `3` units (depending on the units H is expressed in).

Now, we want to `transpile` the circuit to a `Target` backend and actually execute the circuit.

## 5) Under the Hood: A look at DAGCircuit

In [ ]:
# Converting the quantum circuit (Python) to a DAG (Rust)
dag = circuit_to_dag(qc_assigned)

print("Number of DAG nodes:", len(list(dag.op_nodes())))
print("\nTopological order:")
for node in dag.topological_op_nodes():
    print(node.name)

In [ ]:
dag.draw()

In [ ]:
# Remove all barriers
dag.remove_all_ops_named("barrier")
dag.draw()

In [ ]:
# Converting back to a circuit to see changes
new_qc = dag_to_circuit(dag)
new_qc.draw("mpl")

## 6) Define a Custom Target (Hardware Contract)

Though there exist several pre-defined backends (simulators and real QPUs), we have a breif look at how we may define our own custom hardware `Target` backends.

We are making a simple 4-qubit architecture with single qubit Rx/Rz operations and CX operations only.

In [ ]:
target = Target(num_qubits=4)

# Properties of the Rx and Rz gates:
rx_props = {
    (q,): InstructionProperties(duration=25e-9, error=1e-4)
    for q in range(4)
}
rz_props = {
    (q,): InstructionProperties(duration=20e-9, error=5e-5)
    for q in range(4)
}

# Add the Rx and Rz gates (with properties) to the instruction set of our custom Target
target.add_instruction(RXGate(np.pi/6), rx_props)
target.add_instruction(RZGate(np.pi/4), rz_props)

# Add CX gates between neighbouring qubits to the Target instruction set:
cx_props = {
    (0,1): InstructionProperties(duration=300e-9, error=2e-3),
    (1,2): InstructionProperties(duration=300e-9, error=2e-3),
    (2,3): InstructionProperties(duration=300e-9, error=2e-3),
}
target.add_instruction(CXGate(), cx_props)

In [ ]:
target

In [ ]:
target.operation_names

In [ ]:
target.instructions

In [ ]:
coupling_map = target.build_coupling_map()
print(coupling_map)

## 7) Transpilation at Different Optimization Levels

In [ ]:
# Default Qiskit Transpiler Pass Managers (at optimization_level 0/1/2/3)
pm0 = generate_preset_pass_manager(optimization_level=0, target=target)
pm1 = generate_preset_pass_manager(optimization_level=1, target=target)
pm2 = generate_preset_pass_manager(optimization_level=2, target=target)
pm3 = generate_preset_pass_manager(optimization_level=3, target=target)

In [ ]:
# Running the transpiler on the same circuit before and after
t0 = pm0.run(qc_no_meas)
t1 = pm1.run(qc_no_meas)
t2 = pm2.run(qc_no_meas)
t3 = pm3.run(qc_no_meas)

In [ ]:
# An Equivalent Syntax (if you want to use the default Qiskit Pass Managers)
t0 = transpile(qc_no_meas, target=target, optimization_level=0)
t1 = transpile(qc_no_meas, target=target, optimization_level=1)
t2 = transpile(qc_no_meas, target=target, optimization_level=2)
t3 = transpile(qc_no_meas, target=target, optimization_level=3)

In [ ]:
# Effects of optimization:
for i, circ in enumerate([t0,t1, t2, t3]):
    print(f"\nOptimization Level {i}")
    print("Depth:", circ.depth())
    print("Size:", circ.size())
    print("Gate count:", circ.count_ops())

* We can see that higher optimization levels result in better, more efficient circuits (with smaller depth and gate counts)
* In this case, we have a small circuit and `Target` with a very simple gate set (with smaller scope for optimizations).
* For much larger and practical circuits and modern hardware with richer gate sets, the effect of optimization levels in the transpiler is much more drastic.
* Now, let us have a look at how each of the transpiled circuits look...

In [ ]:
t1.draw("mpl")

In [ ]:
dag1 = circuit_to_dag(t1)
dag1.draw()

In [ ]:
t3.draw("mpl")

In [ ]:
dag3 = circuit_to_dag(t3)
dag3.draw()

## 8) Running on a Fake Backend

* Just as an exercize, we defined our own custom (although primitive) Target.
* Now let us transpile and run our circuit on a more realistic, practical backend.

In [ ]:
# Create fake backend
backend = FakeManilaV2()

# Transpile at different optimization levels:
qc_transpiled_0 = transpile(qc_no_meas, backend, optimization_level=0)
qc_transpiled_1 = transpile(qc_no_meas, backend, optimization_level=1)
qc_transpiled_2 = transpile(qc_no_meas, backend, optimization_level=2)
qc_transpiled_3 = transpile(qc_no_meas, backend, optimization_level=3)

Visualizing the effects of transpilation (as earlier):

In [ ]:
# Effects of optimization:
for i, circ in enumerate([qc_transpiled_0,qc_transpiled_1, qc_transpiled_2, qc_transpiled_3]):
    print(f"\nOptimization Level {i}")
    print("Depth:", circ.depth())
    print("Size:", circ.size())
    print("Gate count:", circ.count_ops())

In [ ]:
qc_transpiled_1.draw("mpl")

In [ ]:
qc_transpiled_3.draw("mpl")

Other Visualizations:
1. DAGCircuit representation

In [ ]:
dag3 = circuit_to_dag(qc_transpiled_3)
dag3.draw()

2. Quantum Info

In [ ]:
state = Statevector(qc_transpiled_3)
plot_state_city(state)

In [ ]:
plot_state_hinton(state)

In [ ]:
matrix = DensityMatrix(qc_transpiled_3)
matrix.draw("latex")

In [ ]:
plot_state_paulivec(matrix)

3. Hardware Visualization

In [ ]:
coupling_map = backend.coupling_map
print(coupling_map)

In [ ]:
plot_coupling_map(num_qubits = 133, coupling_map = list(coupling_map), qubit_coordinates=None)

In [ ]:
plot_gate_map(backend)

In [ ]:
plot_error_map(backend)

In [ ]:
plot_circuit_layout(qc_transpiled_3, backend)

4. Visualizing the Pass Manager

In [ ]:
pm = generate_preset_pass_manager(optimization_level=3)
# pass_manager_drawer(pm) {Try to see if it works, sometimes doesn't work on some Window's environments}

In [ ]:
# IF THE DRAWER DOESN'T WORK:
from graphviz import Source
pass_manager_drawer(pm, filename="pass_manager.dot", raw=True)
with open("pass_manager.dot") as f:
    dot_graph = f.read()
Source(dot_graph)

In [ ]:
pm = generate_preset_pass_manager(optimization_level=0)
# pass_manager_drawer(pm) {Try to see if it works, sometimes doesn't work on some Window's environments}

In [ ]:
# IF THE DRAWER DOESN'T WORK
pass_manager_drawer(pm, filename="pass_manager.dot", raw=True)
with open("pass_manager.dot") as f:
    dot_graph = f.read()
Source(dot_graph)

NOW, to actually execute the transpiled quantum circuit, we the `EstimatorV2` Primitive for the Hamiltonian `H` to obtain the average energy level of the system:

In [ ]:
# Define Estimator
estimator = EstimatorV2(mode=backend)
estimator.options.default_shots = 500000

# Define the new Hamiltonian
H_new = H.apply_layout(layout=qc_transpiled_3.layout) # Since the transpiled circuit uses 5 qubits in a different layout 

# Define the PUB (Primitive Unified Block)
# tuple: (circuit, observables, parameter_values{optional})
pub = (qc_transpiled_3, H_new)

In [ ]:
# Execute the job on the FakeManilaV2 Backend:
job = estimator.run([pub])
result = job.result()

# Obtain the expectation value (evs) of the Hamiltonian (average energy level of the system)
energy = result[0].data.evs
print(f"Energy: {energy}")

In [ ]:
# Using the StatevectorEstimator directly:
exact_estimator = StatevectorEstimator()
job = exact_estimator.run([(qc_transpiled_3, H_new)])
exact_energy = job.result()[0].data.evs
print("Exact energy (theoretical):", exact_energy)

* We see this deviation in answers because of hardware noise (Manila is an old processor, ~6 years old).
* We can expect better results using modern hardware with less noise (in fact in the Fault-tolerance regime ~2029 we can just say there is no noise).

Let's try using a noiseless backend (AerSimulator):

In [ ]:
# Pick AerSimulator as the backend (noiseless):
backend = AerSimulator()

In [ ]:
# Rest of the code is the same:
estimator = EstimatorV2(mode=backend)
estimator.options.default_shots = 100000

H_new = H.apply_layout(layout=qc_transpiled_3.layout)
pub = (qc_transpiled_3, H_new)

job = estimator.run([pub])
result = job.result()

energy = result[0].data.evs
print(f"Energy: {energy}")

* As we see, we converge to a better value (much closer to the theoretical).
* For a 4-qubit system it's easy to use an exact simulation, but this becomes exponentially difficult as the qubits increase (In fact, for an $n$-qubit system, we would need a $2^n$ dimensional Hamiltonian matrix for exact estimation using `StatevectorEstimator`).
* So Quantum computers and circuits are required for large, realistic material/chemistry applications.